In [3]:
''' LIBRARIES '''
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import re
import time
from IPython.display import clear_output
import multiprocessing
import threading
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve
import pandas as pd
import numpy as np

# Navigators

In [ ]:
''' NAVIGATOR BETSSON '''
url_bet = 'https://pba.betsson.bet.ar/apuestas-deportivas/futbol'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
#bets = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
bets = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bets.get(url_bet)
bets.maximize_window()
cookie = bets.find_element(By.CSS_SELECTOR, 'button#onetrust-accept-btn-handler')
cookie.click()

print('first sleep')
time.sleep(10)
today = WebDriverWait(bets, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/obg-app-root/div/obg-m-betting-layout-container/obg-m-sportsbook-layout-container/obg-m-sidenav/mat-sidenav-container/mat-sidenav-content/div/div/ng-scrollbar/div/div/div/div/section/ng-component/obg-m-category-container/obg-m-events-master-detail-tabs-container/obg-m-events-master-detail-container/div/div[2]/span[1]')))
today.click()

actions = ActionChains(bets)
print('second sleep')
time.sleep(5)
#show_more= WebDriverWait(bets, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.obg-show-more-less-button')))
#actions.move_to_element(show_more).perform()
#show_more.click()

In [ ]:
''' NAVIGATOR BETWAY'''
bet_way = 'https://betway.bet.ar/es-ar/sports/lve/soccer'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
#bway = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
bway = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bway.get(bet_way)
bway.maximize_window()

upcoming = WebDriverWait(bway, 4).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@collectionitem, "UPCOMING")]')))
upcoming.click()

In [4]:
''' CODERE navigator '''


url_codere = 'https://m.caba.codere.bet.ar/deportes/#/TodayPage'

options = Options()

cod = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
cod.get(url_codere)
cod.maximize_window()

cookie = WebDriverWait(cod, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/ion-app/ion-alert/div/div[3]/button/span')))
cookie.click()
try:
    futbol = WebDriverWait(cod, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="nav"]/page-calendar/ion-grid/ion-row/ion-col/ion-content/div[2]/div[1]/sb-filter/div/div/div[2]/span')))
    futbol.click()
    time.sleep(2)
except:
    print("Didn't click in futbol")
    pass
actions = ActionChains(cod)


# Functions

In [ ]:
def betsson_pre():
    df_bet = pd.DataFrame(columns=['teams', '1_betson', 'X_betson', '2_betson', 'under_betson', 'goals_betson', 'over_betson'])
    for j in range(10):
        for k in range(25):
            try:
                scroll = bets.find_element(By.XPATH, f'/html/body/obg-app-root/div/obg-m-betting-layout-container/obg-m-sportsbook-layout-container/obg-m-sidenav/mat-sidenav-container/mat-sidenav-content/div/div/ng-scrollbar/div/div/div/div/section/ng-component/obg-m-category-container/obg-m-events-master-detail-tabs-container/obg-m-events-master-detail-container/div/obg-accordion-content[{j}]/div/obg-show-more-less/obg-event-table-container/div/obg-event-row-container[{k}]/div/div[1]')
                scroll.location_once_scrolled_into_view
                elemento= bets.find_element(By.XPATH, f'/html/body/obg-app-root/div/obg-m-betting-layout-container/obg-m-sportsbook-layout-container/obg-m-sidenav/mat-sidenav-container/mat-sidenav-content/div/div/ng-scrollbar/div/div/div/div/section/ng-component/obg-m-category-container/obg-m-events-master-detail-tabs-container/obg-m-events-master-detail-container/div/obg-accordion-content[{j}]/div/obg-show-more-less/obg-event-table-container/div/obg-event-row-container[{k}]/div/div[1]').text.split('\n')

                teams = elemento[1]+' '+elemento[2]

                home = elemento[7]
                tie = elemento[9]
                away = elemento[11]

                under = elemento[17]
                goals = int("".join(re.findall(r'\d+', elemento[16])))/10
                over = elemento[15]

                #print(elemento[1]+' '+elemento[2], f'         ||len elemento: {len(elemento)}, j:{j}, k{k}')
                df_bet.loc[len(df_bet.index)] = [teams, home, tie, away, under, goals, over]
            except:
                pass
    for j in range(10):
        for k in range(25):
            try:
                scroll = bets.find_element(By.XPATH, f'/html/body/obg-app-root/div/obg-m-betting-layout-container/obg-m-sportsbook-layout-container/obg-m-sidenav/mat-sidenav-container/mat-sidenav-content/div/div/ng-scrollbar/div/div/div/div/section/ng-component/obg-m-category-container/obg-m-events-master-detail-tabs-container/obg-m-events-master-detail-container/div/obg-accordion-content[{10-j}]/div/obg-show-more-less/obg-event-table-container/div/obg-event-row-container[{25-k}]/div/div[1]')
                scroll.location_once_scrolled_into_view
            except:
                pass
    df_bet.to_csv('CSV\df_betsson_pre.csv')
    return df_bet

In [ ]:
betsson_pre()

In [ ]:
def betway_pre():
    df_bway = pd.DataFrame(columns=['teams', '1_bway', 'X_bway', '2_bway'])
    df_bway2 = pd.DataFrame(columns=['teams', 'under_bway', 'goals_bway', 'over_bway'])
    dia_colapsado = bway.find_elements(By.CSS_SELECTOR, '[collectionitem*="2022"]>div.collapsableHeader[collapsed="true"]')
    for dia in dia_colapsado:
        try:
            dia.click()
            print('click')
            time.sleep(3)
        except:
            pass
    for j in range(20):
        for i in range(10):
            try:
                elemento = bway.find_element(By.XPATH, f'//div[{j}]/div/div/div[@class="oneLineEventItem"][{i}]').text.split('\n')
                teams = elemento[1]

                home = elemento[2]
                tie  = elemento[3]
                away = elemento[4]
                df_bway.loc[len(df_bway.index)] = [teams, home, tie, away]
            except:
                pass
    ''' Clicking to under/over marker '''
    dropdown = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[1]')
    dropdown.click()
    time.sleep(1.5)
    TotalGoals = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div[1]/div[2]')
    TotalGoals.click()

    for j in range(20):
        for i in range(10):
            try:
                elemento = bway.find_element(By.XPATH, f'//div[{j}]/div/div/div[@class="oneLineEventItem"][{i}]').text.split('\n')
                teams = elemento[1]
                goals = elemento[2]
                under = elemento[3]
                over  = elemento[5]
                df_bway2.loc[len(df_bway2.index)] = [teams, under, goals, over]
                
            except:
                pass
    df_bway = pd.merge(df_bway, df_bway2, on='teams')
    dropdown.click()
    time.sleep(1.5)
    HomeXaway = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div[1]/div[1]')
    HomeXaway.click()
    df_bway.to_csv('CSV\df_bway_pre.csv')
    return df_bway

In [ ]:
betway_pre()

In [8]:
def codere_prematch(days=4):
    dias = cod.find_elements(By.CSS_SELECTOR, 'div.sb-calendar-navbar')
    df_cod = pd.DataFrame(columns=['teams', '1_code', 'X_code', '2_code', 'under_cod', 'goals_cod', 'over_cod'])
    for i in range(days+1):
        try:
            dias[i].click()
            print('day click')
            time.sleep(3)
        except:
            pass
        dropdown = cod.find_elements(By.CSS_SELECTOR, 'sb-dropdown.is-collapsable')[:15]
        for drop in dropdown:
            try:
                actions.move_to_element(drop).perform()
                drop.click()
                time.sleep(1)        
            except:
                pass
        partidos = cod.find_elements(By.CSS_SELECTOR, 'sb-grid-item.sb-grid-content--teams')
        try:
            for elemento in partidos:
                partido = elemento.text.split('\n')
                teams = partido[0]+' '+partido[1]

                home = partido[4]
                tie  = partido[6]
                away = partido[8]
                lenn = len(partido)
                under, goals, over = 1, 1, 1
                if len(partido)==14:
                    under = partido[12]
                    goals = ''.join([n for n in partido[9] if n.isdigit()])
                    over  = partido[10]

                df_cod.loc[len(df_cod.index)] = [teams, home, tie, away, under, goals, over]
        except:
            pass
        df_cod.to_csv('CSV\df_cod_pre.csv')
        print(('refresh'))
        cod.refresh()
    return df_cod

In [7]:
codere_prematch()

,teams,1_code,X_code,2_code,under_cod,goals_cod,over_cod
0,Talleres De Córdoba Godoy Cruz,1,"5,75","1,10",1,1,1
1,Boca Juniors Aldosivi,"1,37","4,40",9,"1,83",25,"1,95"
2,Patronato River Plate,"4,50","3,60","1,75",2,25,"1,80"
3,Csd San Martin Ca Atlas,1,"3,20","1,30",1,1,1
4,Cs Italiano Ca Ferrocarril Midland,0,"1,90","1,80",1,1,1
5,Sportivo Ac Las Parejas Boca Unidos,0,"2,70","1,80",1,1,1
6,Gimnasia Y Tiro Gimnasia Y Esgrima De Concepci...,0,"1,09","6,25",1,1,1
7,Cd Argentino Monte Maiz Club Sportivo Estudiantes,2,18,"6,75",1,1,1
8,Ca Ferro Carril Oeste General Pico Club Villa ...,0,"1,60","2,90",1,1,1
9,Sportivo Penarol San Juan Club Ciudad De Bolívar,0,"2,80","2,60",1,1,1
